In [26]:
# !pip install openai
# !pip install langchain
# !pip install langchain-openai
# !pip install docarray

Ok, so far we've discussed the basics of agents, and looked at practical examples of implementing them in Python, from a naive approach where we give tools inside the prompt to the model and ask it to generate function calls from there, to using frameworks like openai or langchain in combination with function calling to better optimize and struture these calls to the necessary tools. 

Now, let's look at in practice how to put things together to build cool agents that can perform interesting and productive tasks.

The framework we'll use to do that is [LangChain](https://python.langchain.com/docs/get_started/introduction).

So, before we dive into agents, let's quickly take a look at this framework to understand how it allows for building these complex functionalities.

# Introduction to LangChain 

Working with LLMs involves in one way or another working with a specific type of abstraction: "Prompts".

However, in the practical context of day-to-day tasks we expect LLMs to perform, these prompts won't be some static and dead type of abstraction. Instead we'll work with dynamic prompts re-usable prompts.

# Lanchain

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a framework that allows you to connect LLMs together by allowing you to work with modular components like prompt templates and chains giving you immense flexibility in creating tailored solutions powered by the capabilities of large language models.


Its main features are:
- **Components**: abstractions for working with LMs
- **Off-the-shelf chains**: assembly of components for accomplishing certain higher-level tasks

LangChain facilitates the creation of complex pipelines that leverage the connection of components like chains, prompt templates, output parsers and others to compose intricate pipelines that give you everything you need to solve a wide variety of tasks.

At the core of LangChain, we have the following elements:

- Models
- Prompts
- Output parsers

**Models**

Models are nothing more than abstractions over the LLM APIs like the ChatGPT API.​

In [27]:
# uncomment this if running locally
# from dotenv import load_dotenv

# load_dotenv()

# Or if you are in Colab, uncoment below and add your api key
# import os
# os.environ["OPENAI_API_KEY"] = "your-api-key"

In [28]:
from langchain.llms import OpenAI
from langchain_openai.chat_models import ChatOpenAI
import os

# os.environ["OPENAI_API_KEY"]=""
chat_model = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-3.5-turbo")

You can predict outputs from both LLMs and ChatModels:

In [29]:
chat_model.invoke("hi!")
# Output: "Hi"

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-52bb56f7-e708-43e1-94f8-3b661d015bea-0')

In [30]:
chat_model.invoke("What is the best part about using LLMs with tools?")

AIMessage(content='The best part about using Language Model as a Service (LLMs) with tools is that it allows for improved accuracy and efficiency in various natural language processing tasks. By leveraging pre-trained language models, users can quickly generate high-quality text, perform sentiment analysis, automate content creation, and more, without the need for extensive training or fine-tuning. Additionally, integrating LLMs with tools can help developers and businesses streamline their workflows, reduce manual effort, and enhance the overall user experience.', response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 20, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a4e52950-8576-46ca-a6ee-966a72af6298-0')

Basic components are:

- Models
- Prompt templates
- Output parsers

In [31]:
from langchain.prompts import (
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)
from langchain.schema.output_parser import StrOutputParser

Comprehensive example:

In [32]:
# Define the system message
system_template = """You are a helpful AI assistant who is weirdly obsessed with pancakes. 
Your role is to provide helpful information to users, but you always make a random comment
at the end about pancakes.
"""

In [33]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [34]:
# Define the human message 
human_template = "Hello, {subject}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Combine the system and human messages into a chat prompt
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

# Format the chat prompt with a subject
formatted_prompt = chat_prompt.format_prompt(subject="AI!")
print(formatted_prompt)

messages=[SystemMessage(content='You are a helpful AI assistant who is weirdly obsessed with pancakes. \nYour role is to provide helpful information to users, but you always make a random comment\nat the end about pancakes.\n'), HumanMessage(content='Hello, AI!')]


Template Example

In [35]:
prompt = ChatPromptTemplate.from_template("Show me 5 names for a company that makes: {product}")

In [36]:
prompt

ChatPromptTemplate(input_variables=['product'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['product'], template='Show me 5 names for a company that makes: {product}'))])

In [37]:
type(prompt)

langchain_core.prompts.chat.ChatPromptTemplate

In [38]:
prompt.format(product="chairs")

'Human: Show me 5 names for a company that makes: chairs'

In [39]:
chain = prompt | chat_model

output = chain.invoke({"product": "animal"})
output

AIMessage(content='1. Creature Creations Co.\n2. Furry Friends Forge\n3. Beastly Builders Inc.\n4. Wild Wonders Workshop\n5. Critter Crafts Company', response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 19, 'total_tokens': 53}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f7b34d7a-b6eb-4e62-840d-06bd82d55cd4-0')

In [40]:
from IPython.display import Markdown

Markdown(output.content)

1. Creature Creations Co.
2. Furry Friends Forge
3. Beastly Builders Inc.
4. Wild Wonders Workshop
5. Critter Crafts Company

In [41]:
chat_model.invoke("I am teaching a live-training about LLMs!")

AIMessage(content="That's great! LLMs, or Master of Laws degrees, are specialized postgraduate degrees for students who have already completed a law degree. They allow students to deepen their knowledge in a specific area of law, such as international law, tax law, or intellectual property law. During your live training, you can cover the different types of LLM programs available, the admissions process, career opportunities for LLM graduates, and any other relevant information. Make sure to engage your audience with interactive activities and encourage questions to ensure they are getting the most out of the training. Good luck with your session!", response_metadata={'token_usage': {'completion_tokens': 120, 'prompt_tokens': 18, 'total_tokens': 138}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b46c3c5e-909c-44e3-89a9-f0267b951387-0')

In [42]:
from langchain.schema import HumanMessage

text = "What would be a good dog name for a dog that loves to nap?"
messages = [HumanMessage(content=text)]

chat_model.invoke(messages)

AIMessage(content='Snooze', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 22, 'total_tokens': 25}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4e3f9166-1640-4f23-8842-835ea13a6820-0')

At this point let's stop and take a look at what this code would look like if we were using the openai api directly instead.

Let's understand what is going on.

Instead of writing down the human message dictionary for the openai API as you would do normally using the the original API, langchain is giving you an abstraction over that message through the class
`HumanMessage()`, as well as an abstraction over the loop for multiple predictions through the .`invoke()` method.

Now, why is that an useful thing?

Because it allows you to work at a higher level of experimentation and orchestration with the blocks of that make up a workflow using LLMs.

By making it easier to create predictions of multiple messages for example, you can experiment with different human message prompts faster and therefore get to better and more efficient results faster without having to write a lot of boilerplate.

**Prompts**

The same works for prompts. Now, prompts are pieces of text we feed to LLMs, and LangChain allows you to work with prompt templates.

Prompt Templates are useful abstractions for reusing prompts and they are used to provide context for the specific task that the language model needs to complete. 

A simple example is a `PromptTemplate` that formats a string into a prompt:

In [43]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")
prompt.format(activity="sleeping")
# Output: "What is a good dog name for a dog that loves to nap?"

'What is a good dog name for a dog that loves to sleeping?'

**Output Parsers**

OutputParsers convert the raw output from an LLM into a format that can be used downstream. Here is an example of an OutputParser that converts a comma-separated list into a list:

In [44]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")
# Output: ['hi', 'bye']

['hi', 'bye']

In [45]:
from langchain.schema import StrOutputParser

print(StrOutputParser().parse(output))

content='1. Creature Creations Co.\n2. Furry Friends Forge\n3. Beastly Builders Inc.\n4. Wild Wonders Workshop\n5. Critter Crafts Company' response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 19, 'total_tokens': 53}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-f7b34d7a-b6eb-4e62-840d-06bd82d55cd4-0'


This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to an LLM, and then pass the output through an output parser.

Ok, so these are the basics of langchain. But how can we leverage these abstraction capabilities inside our LLM app application?

Now, to put everything together LangChain allows you to build something called "chains", which are components that connect prompts, llms and output parsers into a building block that allows you to create more interesting and complex functionality.

Let's look at the example below:

In [46]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")

chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=prompt,
)
chain.invoke("sleep")

{'activity': 'sleep', 'text': 'Snuggles'}

So, what the chain is doing is connecting these basic components (the LLM and the prompt template) into
a block that can be run separately. The chain allows you to turn workflows using LLLMs into this modular process of composing components.

Now, the newer versions of LangChain have a new representation language to create these chains (and more) known as LCEL or LangChain expression language, which is a declarative way to easily compose chains together. The same example as above expressed in this LCEL format would be:

In [47]:
chain = prompt | ChatOpenAI()

chain.invoke({"activity": "sleep"})

AIMessage(content='Snooze', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 21, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-011a7063-734e-4000-9859-2eeccdc43823-0')

In [48]:
chain = prompt | ChatOpenAI() | StrOutputParser()

chain.invoke({"activity": "sleep"})

'Snuggles'

Notice that now the output is an `AIMessage()` object, which represents LangChain's way to abstract the output from an LLM model like ChatGPT or others.

These building blocks and abstractions that LangChain provides are what makes this library so unique, because it gives you the tools you didn't know you need it to build awesome stuff powered by LLMs.

In [49]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from IPython.display import Markdown


model = ChatOpenAI(temperature=0)

prompt_template_string = "Name 5 concepts related to this: {concept}.\
    The output should be in bullet points."
prompt = ChatPromptTemplate.from_template(template=prompt_template_string)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

Markdown(chain.invoke({"concept": "probability distribution"}))

- Normal distribution
- Binomial distribution
- Poisson distribution
- Exponential distribution
- Uniform distribution

In [50]:
llm.invoke("Hi! Tell me a joke about an instructor who is so childish, he makes examples where he calls himself the agent-master")

AIMessage(content='Sure, here\'s a joke for you:\n\nWhy did the instructor who called himself the "agent-master" always get in trouble during class?\n\nBecause every time he gave an example, he ended up playing with the "action figures" instead of teaching the lesson!', response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 31, 'total_tokens': 82}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_c4e5b6fa31', 'finish_reason': 'stop', 'logprobs': None}, id='run-55ed1706-9e4e-4559-b8ab-2a89597a2979-0')